In [1]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sn
import utils
from matplotlib.backends.backend_pdf import PdfPages
from tqdm.notebook import tqdm

plt.style.use("ggplot")
custom_palette = [
    "#E24A33",
    "#348ABD",
    "#988ED5",
    "#777777",
    "#FBC15E",
    "#8EBA42",
    "#FFB5B8",
    "#17BECF",
]
plt.rcParams["axes.prop_cycle"] = plt.cycler(color=custom_palette)

# Sensitivity to eutrophication

# Part 3: TEOTIL results

Notebook 02 used the Vann-Nett API to get the regine ID and vassdragsområde number for each waterbody of interest. For lake and river waterbodies, I have made some manual adjustments to the regine IDs, because the ones assigned in Vann-Nett seem incorrect. For coastal waterbodies, Andre just wants total discharges from TEOTIL for each vassdragsområde, which I assume are correctly assigned in Vann-Nett.

This notebook loops over the list of waterbodies from Miljødirektoratet and performs the following steps for each:

 1. For coastal waterbodies, get the vassdragsområde ID; for river and lake waterbodies, get the regine ID.
    
 2. Extract results from TEOTIL2 and TEOTIL3 for the vassdragsområde/regine of interest. TEOTIL2 results cover the period from 1996 to 2020, while TEOTIL3 covers 2013 to 2023.
    
 3. Calculate a bias-correction factor for TEOTIL2 based on the period of model overlap (2013 to 2022). For each parameter and vassdragsområde/regine, calculate the total mean load simulated by each model and estimate the correction factor as $\frac{TEO3 \ Load}{TEO2 \ Load}$.
    
 4. Bias-correct the TEOTIL2 results for 1996 to 2012 by scaling them using the correction factor.
    
 5. Create a single modelled series for each parameter and vassdragsområde/regine by combining the bias-corrected TEOTIL2 data from 1996 to 2012 with TEOTIL3 output from 2013 to 2023.
    
 6. Creates plots showing time series of annual discharges split by source for each vassdragsområde/regine.

## 1. User input

In [2]:
# Pars of interest
par_list = ["n", "p"]

# Periods of interest
teo2_st_yr = 1996
teo2_end_yr = 2022
teo3_st_yr = 2013
teo3_end_yr = 2023
bias_st_yr = 2013
bias_end_yr = 2022

# Extra options for TEOTIL3
nve_data_yr = 2024
agri_loss_model = "annual"

# File paths
wb_xl_path = r"../data/mdir_waterbodies_with_vann-nett_info_patched.xlsx"
res_xl_path = r"../data/teotil_results_for_waterbodies.xlsx"
pdf_path = r"../images/teotil_results_for_waterbodies.pdf"

## 2. Waterbodies of interest

In [3]:
# Read the waterbody list from notebook 02 (with manually patched regine IDs)
wb_df = pd.read_excel(wb_xl_path)
wb_df.head()

,VannforekomstID,Id,Navn.vannforekomst,Breddegrad,Lengdegrad,Zero,Areal (km2),Type,Opphold.bunnvann,NIVA.data,...,regine_teo2,vassom_no,vassom_name,national_water_type,salinity_class,O2_value,O2_unit,O2_sample_count,O2_from_year,O2_to_year
0,0120000034-C,C01,Merdø - Hasseltangen,58.3998,8.7617,0.0,8.390,C,Kort,NaN,...,019.211,19,Arendalsvassdraget/kyst Moland-Homborsund,S2,Skagerak (> 25),5.254020,ml/l,14.0,2020.0,2021.0
1,0121000300-1-C,C02,Grosfjorden - indre,58.3196,8.5912,0.0,5.574,C,Oksygenfattig (og moderat),NaN,...,019.21610,19,Arendalsvassdraget/kyst Moland-Homborsund,NaN,Polyhaline (18 - 30),3.060000,NaN,NaN,2015.0,2015.0
2,0121010500-1-C,C03,Lillesandsfjorden,58.2450,8.3883,0.0,2.064,C,Moderat,Ja,...,020.2210,20,Tovdalsvassdraget/Lillesand kommune,S3,Skagerak (> 25),5.417373,ml/l,21.0,2019.0,2021.0
3,0130010301-2-C,C04,Østergapet - indre,58.1141,8.0348,0.0,22.523,C,Kort,NaN,...,021.220,21,Otra/Kristiansand og Flekkerøy,S2,Skagerak (> 25),NaN,NaN,NaN,NaN,NaN
4,0131010200-C,C05,Høllefjorden,58.0703,7.8092,0.0,1.729,C,Moderat,NaN,...,022.12,22,Mandalselva/kyst Flekkerøy-Mandal by,S3,Skagerak (> 25),NaN,NaN,NaN,NaN,NaN


## 3. Get TEOTIL results

In [4]:
df_list = []
for idx, row in tqdm(wb_df.iterrows(), total=len(wb_df)):
    # Get waterbody details
    wb_id = row["VannforekomstID"]
    wb_type = row["Type"]

    # Use vassom for coastal WBs, otherwise regine
    if wb_type == "C":
        teo2_reg_id = teo3_reg_id = f"{row['vassom_no']:03d}."
    elif wb_type in ("R", "L"):
        teo2_reg_id = row["regine_teo2"]
        teo3_reg_id = row["regine"]
    else:
        raise ValueError(f"Could not parse waterbody type for '{wb_id}'.")

    # Get TEOTIL results
    teo2_df = utils.get_teotil2_results_for_regine(teo2_st_yr, teo2_end_yr, teo2_reg_id)
    teo2_df['regine'] = teo3_reg_id
    teo3_df = utils.get_teotil3_results_for_regine(
        teo3_st_yr,
        teo3_end_yr,
        teo3_reg_id,
        agri_loss_model,
        nve_data_yr,
    )

    # Process data for each area
    for par in par_list:
        # Standardise and aggregate output params
        teo2_par_df = utils.aggregate_parameters(teo2_df, par, "teotil2")
        teo3_par_df = utils.aggregate_parameters(teo3_df, par, "teotil3")

        # Get TEOTIL2 results for period matching TEOTIL3 results
        # and calculate average for each vassom
        teo2_avg_par_df = (
            teo2_par_df.query("@bias_st_yr <= `År` <= @bias_end_yr")
            .drop(columns="År")
            .groupby("regine")
            .mean()
        )
        teo3_avg_par_df = (
            teo3_par_df.query("@bias_st_yr <= `År` <= @bias_end_yr")
            .drop(columns="År")
            .groupby("regine")
            .mean()
        )

        # Bias correct based on period for 2013 to 2022 for each vassom
        bias_df = (teo3_avg_par_df / teo2_avg_par_df).fillna(0).reset_index()
        bias_df.replace([np.inf, -np.inf], 1, inplace=True)
        bias_df = bias_df.set_index("regine").clip(lower=0, upper=10).reset_index()
        teo2_par_df = teo2_par_df.query("@teo2_st_yr <= `År` < @teo3_st_yr")
        teo2_par_df = pd.merge(
            teo2_par_df, bias_df, how="left", on="regine", suffixes=("", "_fac")
        )
        for src in bias_df.columns:
            if src != "regine":
                teo2_par_df[src] = teo2_par_df[src] * teo2_par_df[f"{src}_fac"]
                del teo2_par_df[f"{src}_fac"]

        # Merge bias-corrected TEOTIL2 data with TEOTIL3 output
        par_df = (
            pd.concat([teo2_par_df, teo3_par_df], axis="rows")
            .sort_values(["regine", "År"])
            .reset_index(drop=True)
        )

        # Calculate totals
        par_df["Menneskeskapt"] = (
            par_df["Akvakultur"]
            + par_df["Jordbruk"]
            + par_df["Avløp"]
            + par_df["Industri"]
            + par_df["Bebygd"]
        )
        par_df["Totalt"] = par_df["Menneskeskapt"] + par_df["Bakgrunn"]
        par_df.rename(columns={"regine": "Regine"}, inplace=True)
        par_df["Par"] = f"TOT{par.upper()}"
        par_df["Vannforekomst_ID"] = wb_id
        par_df.columns = [
            (
                f"{col}_tonn"
                if col not in ("Vannforekomst_ID", "Regine", "År", "Par")
                else col
            )
            for col in par_df.columns
        ]
        id_cols = ["Vannforekomst_ID", "Regine", "Par", "År"]
        src_cols = [col for col in par_df.columns if col not in id_cols]
        par_df = par_df[id_cols + src_cols]
        par_df[src_cols] = par_df[src_cols].round(3)
        df_list.append(par_df)
res_df = pd.concat(df_list, axis="rows").reset_index(drop=True)
res_df

  0%|          | 0/153 [00:00<?, ?it/s]

,Vannforekomst_ID,Regine,Par,År,Akvakultur_tonn,Jordbruk_tonn,Avløp_tonn,Industri_tonn,Bebygd_tonn,Bakgrunn_tonn,Menneskeskapt_tonn,Totalt_tonn
0,0120000034-C,019.,TOTN,1996,0.0,386.777,221.827,0.000,77.732,625.086,686.337,1311.422
1,0120000034-C,019.,TOTN,1997,0.0,379.972,218.290,0.000,77.732,742.861,675.994,1418.855
2,0120000034-C,019.,TOTN,1998,0.0,373.310,208.092,19.753,77.732,824.962,678.887,1503.849
3,0120000034-C,019.,TOTN,1999,0.0,366.533,222.500,6.884,77.732,920.273,673.649,1593.922
4,0120000034-C,019.,TOTN,2000,0.0,359.888,192.397,6.884,77.732,1107.966,636.901,1744.867
...,...,...,...,...,...,...,...,...,...,...,...,...
8563,050-106-R,050.BA,TOTP,2019,0.0,0.043,0.006,0.000,0.019,0.125,0.068,0.193
8564,050-106-R,050.BA,TOTP,2020,0.0,0.073,0.058,0.000,0.048,0.322,0.179,0.501
8565,050-106-R,050.BA,TOTP,2021,0.0,0.056,0.026,0.000,0.026,0.176,0.108,0.284
8566,050-106-R,050.BA,TOTP,2022,0.0,0.068,0.015,0.000,0.036,0.239,0.119,0.358


In [5]:
# Save to Excel
with pd.ExcelWriter(res_xl_path) as writer:
    wb_df.to_excel(writer, sheet_name="Waterbodies", index=False)
    res_df.to_excel(writer, sheet_name="TEOTIL_Results", index=False)

In [6]:
# Remove '_tonn' suffix from res_df
res_df.columns = [
    (col.split("_")[0] if col.endswith("_tonn") else col) for col in res_df.columns
]

In [7]:
# Create summary PDF with one page per vassdragsområde/regine
sources = [
    "Akvakultur",
    "Jordbruk",
    "Avløp",
    "Industri",
    "Bebygd",
    "Bakgrunn",
]

with PdfPages(pdf_path) as pdf:
    for wb_id, wb_df in res_df.groupby("Vannforekomst_ID"):
        reg_id = wb_df["Regine"].iloc[0]

        # Setup plots
        fig, axs = plt.subplots(
            5, 2, figsize=(10, 12), gridspec_kw={"height_ratios": [1, 0.2, 1, 0.2, 1]}
        )
        fig.delaxes(axs[1, 0])
        fig.delaxes(axs[1, 1])
        fig.delaxes(axs[3, 0])
        fig.delaxes(axs[3, 1])

        if reg_id.endswith("."):
            fig.suptitle(f"{wb_id} (vassdragsområde {reg_id[:-1]})\n", fontsize=16)
        else:
            fig.suptitle(f"{wb_id} (regine {reg_id})\n", fontsize=16)

        for i, par in enumerate(["TOTN", "TOTP"]):
            par_df = wb_df.query("Par == @par").copy()
            par_df = par_df.drop(
                columns=["Vannforekomst_ID", "Regine", "Par"]
            ).set_index("År")

            # Line chart
            par_df.plot(ax=axs[0, i], marker="o", legend=False)
            axs[0, i].axvline(2013, c="k", ls="--", lw=1)
            axs[0, i].set_title(par)
            axs[0, i].set_xlabel("")
            axs[0, i].set_ylabel(f"{par} (tonn)")

            # Stacked bar chart
            par_df[sources].plot(kind="bar", stacked=True, ax=axs[2, i], legend=False)
            axs[2, i].set_title(par)
            axs[2, i].set_xlabel("")
            axs[2, i].set_ylabel(f"{par} (tonn)")

            # Horizontal bar chart
            df_period = par_df[
                (par_df.index >= teo3_st_yr) & (par_df.index <= teo3_end_yr)
            ][sources]
            total_sources = df_period[sources].sum()
            percentages = (total_sources / total_sources.sum()) * 100
            percentages = percentages.sort_values()
            percentages.plot(kind="barh", ax=axs[4, i])
            axs[4, i].set_title(
                f"{par} i prosent\n(gjennomsnitt {teo3_st_yr}-{teo3_end_yr})"
            )
            axs[4, i].set_xlabel("Prosent")
            axs[4, i].set_ylabel("")
            for index, value in enumerate(percentages):
                axs[4, i].text(value, index, f"{value:.1f}%", va="center")

        # Add legends below the specified subplots
        handles, labels = axs[0, 0].get_legend_handles_labels()
        fig.legend(
            handles, labels, loc="upper center", bbox_to_anchor=(0.5, 0.69), ncol=3
        )

        handles, labels = axs[2, 0].get_legend_handles_labels()
        fig.legend(
            handles, labels, loc="upper center", bbox_to_anchor=(0.5, 0.34), ncol=3
        )

        plt.tight_layout()
        pdf.savefig(fig)
        plt.close(fig)